In [1]:
"""
Dataset From
https://www.kaggle.com/datasets/volkandl/car-brand-logos?resource=download
Local
"""
import matplotlib.pyplot as plt
import matplotlib
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import random
import torchvision
import pretrainedmodels

matplotlib.style.use('ggplot')

from sklearn.model_selection import train_test_split
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision import models, datasets
from torchvision.datasets import ImageFolder

SEED=42

def seed_everything(SEED):
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

seed_everything(SEED=SEED)

In [2]:
class_list =[
    ['acura',173250000],
    ['alfaromeo',82500000],
    ['buick',59400000],
    ['cadillac',110000000],
    ['dodge',88000000],
    ['fiat',33000000],
    ['opel',44000000]
]

In [3]:
LOGODICT = ["acura", "alfaromeo", "buick", "cadillac", "dodge", "fiat", "opel"]

In [4]:
dataset_path = "F:/프로젝트/CarLogoDetection/data/data"
train_val_split_ratio = 0.8
learning_rate = 0.001
num_epoch = 50
batch_size = 128

trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),  # 50% 확률로 좌우 반전
    transforms.RandomVerticalFlip(p=0.5),  # 50% 확률로 상하 반전
    transforms.ToTensor(),
])

dataset = ImageFolder(root=dataset_path, transform=trans)

In [5]:
class_num = len(dataset.classes)  # or len(dataset.class_to_idx)

print(f"🔹 총 클래스 개수: {class_num}")


🔹 총 클래스 개수: 14


In [6]:
dataset.classes

['acura',
 'alfaromeo',
 'buick',
 'cadillac',
 'dodge',
 'fiat',
 'hyundai',
 'lexus',
 'mazda',
 'mercedes',
 'opel',
 'skoda',
 'toyota',
 'volkswagen']

In [7]:
train_data, test_data = train_test_split(dataset, test_size = 0.2,random_state = 42)

In [8]:
train_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
teat_loader = torch.utils.data.DataLoader(test_data,batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

In [9]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=True)

In [10]:
import torch
from torch import nn
import torchvision.models as models


class ResNet34(nn.Module):
    def __init__(self, pretrained=True):
        super(ResNet34, self).__init__()

        self.class_num = class_num

        if pretrained:
            self.model = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
            print("Pretrained model loaded")
        else:
            self.model = models.resnet34(weights=None)
            print("Pretrained model loading failed, using untrained model")

        # **🔥 기존 fc (1000개 클래스) → 7개 클래스 변경**
        self.model.fc = nn.Linear(512, self.class_num)

    def forward(self, x):
        return self.model(x)  # 모델 그대로 사용 가능


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = ResNet34(pretrained=True).to(device)

Pretrained model loaded


In [11]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
#train
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()

    if i % 10 == 0:
        print(loss)

tensor(0.6628, grad_fn=<NllLossBackward0>)
tensor(0.0863, grad_fn=<NllLossBackward0>)
tensor(0.0190, grad_fn=<NllLossBackward0>)
tensor(0.0845, grad_fn=<NllLossBackward0>)
tensor(0.0003, grad_fn=<NllLossBackward0>)


Accuracy of Test Data: 86.1328125


In [13]:
torch.save(model.state_dict(), 'model')

In [14]:
import torch
import PIL
# 모델 불러오기
model = ResNet34(pretrained=True).to(device)
model.load_state_dict(torch.load("F:\프로젝트\CarLogoDetection\model"))  #학습한 모델 사용

#test
correct = 0
total = 0

with torch.no_grad():
  for image,label in test_loader:
      x = image.to(device)
      y_= label.to(device)
      output = model.forward(x)
      _,output_index = torch.max(output,1)
      total += label.size(0)
      correct += (output_index == y_).sum().float()

  print("Accuracy of Test Data: {}".format(100*correct/total))



<>:5: SyntaxWarning: invalid escape sequence '\C'
<>:5: SyntaxWarning: invalid escape sequence '\C'
C:\Users\nahyu\AppData\Local\Temp\ipykernel_9896\684290289.py:5: SyntaxWarning: invalid escape sequence '\C'
  model.load_state_dict(torch.load("F:\프로젝트\CarLogoDetection\model"))  #학습한 모델 사용


Pretrained model loaded
Accuracy of Test Data: 88.671875


In [ ]:
# 새로운 이미지 로드 및 전처리
image_path = "/content/tester.png"  # 사용할 이미지 파일 업로드 후
image = PIL.Image.open(image_path).convert("RGB")
input_image = trans(image).unsqueeze(0)

if torch.cuda.is_available():
    input_image = input_image.to('cuda')

# 모델 입력 및 예측
output = model(input_image)

# 예측 결과 확인
predicted_label = torch.argmax(output)

print("예측된 라벨:", predicted_label)

In [ ]:
predicted_label.cpu()
k = predicted_label.item()

In [ ]:
com = class_list[k]

print("이 차량 정보: " ,com[0],"사의 차량으로", "최대",com[1],"원 입니다.")